In [57]:
import random
import numpy as np
import pandas as pd
import time

In [58]:
# Se crea la matriz de pesos y recompensas de los cargamentos
weight_list = [61, 58, 92, 50, 108, 83, 93, 101, 54, 50, 72, 51, 100, 108, 91, 112, 66, 58, 110, 73]
reward_list = [1100, 1147, 1442, 1591, 1078, 1385, 1777, 1196, 1753, 1371, 1517, 1675, 1193, 1177, 1365, 1143, 1314, 1526, 1470, 1605]

In [59]:
#Esta función calcula el peso de cada agente en la población
def calc_weight(chromosome, weight):
        return sum([chromosome[i] * weight[i] for i in range(len(chromosome))])
    
def calc_reward(chromosome, reward):
    return sum([chromosome[i] * reward[i] for i in range(len(chromosome))])

def calc_fitness(weight,reward):
    max_weight = 800
    if weight > max_weight:
        reward_per_weight = [reward_element / weight_element for reward_element, weight_element in zip(reward_list, weight_list)]
        penalty_per_unit = np.mean(reward_per_weight)
        total_penalty = (weight - max_weight) * penalty_per_unit + 1300
        
        return reward - total_penalty
    else:
        return reward

In [60]:
# Se crea la clase de agente, cuenta con sus cromosomas, peso y recompensa
class Agent():
    def __init__(self, weight_list, reward_list, num_genes=20):
        self.chromosome = [bool(random.choice([0, 1])) for _ in range(num_genes)]
        self.num_genes = num_genes
        self.weight = calc_weight(self.chromosome, weight_list)
        self.reward = calc_reward(self.chromosome, reward_list)
        self.fitness = calc_fitness(self.weight, self.reward)
                
        
    #Se crea la función de mutar el cual elige un gen al azar y lo cambia
    def mutate(self, weight_list, reward_list):
        is_mutated = [random.random() for _ in range(self.num_genes)]
        self.chromosome = [not self.chromosome[i] if is_mutated[i] < 0.05 else self.chromosome[i] for i in range(self.num_genes)]
        self.weight = calc_weight(self.chromosome, weight_list)
        self.reward = calc_reward(self.chromosome, reward_list)
        self.fitness = calc_fitness(self.weight, self.reward)

In [61]:
#Se toma la mitad de la población con mejor fitness
def selection(population):
    sorted_population = sorted(population, key=lambda x: x.fitness, reverse=True)
    return sorted_population[:len(population)//2]
    
#Se realiza el cruce de los cromosomas de los agentes
def crossover(population):
    offspring = []
    
    # Convert the elements list to a numpy array
    arpopulation = np.array(population)
    
    # Shuffle the elements using numpy's shuffle
    np.random.shuffle(arpopulation)

    # Check if the number of elements is even
    if len(arpopulation) % 2 != 0:
        raise ValueError("The number of elements must be even to form pairs.")
    
    # Reshape the array into pairs
    arparents = arpopulation.reshape(-1, 2)
    
    # Convert the numpy array of pairs back to a list of lists
    parents = arparents.tolist()
    
    for parent in parents:
        parent1 = parent[0] 
        parent2 = parent[1]
        child1 = Agent(weight_list, reward_list)
        child2 = Agent(weight_list, reward_list)
        child3 = Agent(weight_list, reward_list)
        child4 = Agent(weight_list, reward_list)
        child1.chromosome = parent1.chromosome[:10] + parent2.chromosome[10:]
        child2.chromosome = parent2.chromosome[:10] + parent1.chromosome[10:] 
        child3.chromosome = parent2.chromosome[10:] + parent2.chromosome[:10]
        child4.chromosome = parent1.chromosome[10:] + parent1.chromosome[:10]
        childs = [child1, child2, child3, child4]
        
        mutated_childs = mutation(childs)
        mutated_childs = sorted(mutated_childs, key=lambda x: x.fitness)
        offspring.extend(mutated_childs)
        
    return offspring
# Se realiza la probabilidad de mutación
def mutation(population):
    for entity in population:
        entity.mutate(weight_list, reward_list)
    return population

In [62]:
performance = {}

# Se inicializa la población
population_size = 300
population = [Agent(weight_list,reward_list, num_genes=20) for _ in range(population_size)]
counter = 0
past_agent = 0
epoch = 0
#Número de epocas
all_time_best_agent = max(population, key=lambda x: x.fitness)

while counter < 10:
    start_time = time.time()
    epoch += 1
    purged_population = selection(population)
    babies = crossover(purged_population)
    while len(babies) < len(population)//2:
        babies.append(crossover(purged_population))
    babies = babies[:len(population)//2]
    purged_population.extend(babies)
    population = purged_population
    # print(len(population))
    #Look for the best agent
    current_best_agent = max(population, key=lambda x: x.fitness)
    if current_best_agent.fitness <= all_time_best_agent.fitness:
        counter += 1
    else:
        counter = 0
        all_time_best_agent = current_best_agent

    end_time = time.time()

    stats = {
        'Epoch': epoch,
        'Best Agent Fitness': current_best_agent.fitness,
        'Best Agent Reward': current_best_agent.reward,
        'Best Agent Weight': current_best_agent.weight,
        'Execution Time': end_time - start_time
    }

    performance[epoch] = stats

    # print("Epoca:"+ str(epoch))
    # # print("Counter:"+ str(counter))
    # print("Este es el fitness mejor agente: " + str(current_best_agent.fitness))
    # print("Este es el reward mejor agente: " + str(current_best_agent.reward))
    # print("Este es el peso mejor agente: " + str(current_best_agent.weight))

print("RESULTADOS FINALES")
print("Este es el fitness mejor agente: " + str(all_time_best_agent.fitness))
print("Este es el reward mejor agente: " + str(all_time_best_agent.reward))
print("Este es el peso mejor agente: " + str(all_time_best_agent.weight))
print("Este es el cromosoma mejor agente: " + str([index + 1 for index, value in enumerate(all_time_best_agent.chromosome) if value == 1]))


RESULTADOS FINALES
Este es el fitness mejor agente: 18103
Este es el reward mejor agente: 18103
Este es el peso mejor agente: 800
Este es el cromosoma mejor agente: [2, 3, 4, 6, 7, 9, 10, 11, 12, 17, 18, 20]


In [63]:
pd.DataFrame(performance).T

,Epoch,Best Agent Fitness,Best Agent Reward,Best Agent Weight,Execution Time
1,1.0,16476.0,16476.0,754.0,0.049474
2,2.0,16675.0,16675.0,783.0,0.086965
3,3.0,16675.0,16675.0,783.0,0.023188
4,4.0,16675.0,16675.0,783.0,0.024022
5,5.0,16675.0,16675.0,783.0,0.020015
6,6.0,16718.0,16718.0,717.0,0.020158
7,7.0,17157.0,17157.0,784.0,0.019292
8,8.0,17818.0,17818.0,778.0,0.019034
9,9.0,17818.0,17818.0,778.0,0.023523
10,10.0,17818.0,17818.0,778.0,0.021978
